### Funções para o RBF

In [2]:
from __future__ import division
import random
import math
from Pattern import Pattern
from Data import Data
import numpy as np


class RBFNetwork:
    def __init__(self, no_of_input, no_of_hidden, no_of_output, data):
        self.no_of_input = no_of_input
        self.no_of_hidden = no_of_hidden
        self.no_of_output = no_of_output
        self.data = data
        self.input = np.zeros(self.no_of_input)
        self.centroid = np.zeros((self.no_of_hidden, self.no_of_input))
        self.sigma = np.zeros(self.no_of_hidden)
        self.hidden_output = np.zeros(self.no_of_hidden)
        self.hidden_to_output_weight = np.zeros((self.no_of_hidden, self.no_of_output))
        self.output = np.zeros(self.no_of_output)
        self.output_bias = np.zeros(self.no_of_output)
        self.actual_target_values = []
        self.total = 0
        self.learningRate = 0.0262
        self.setup_center()
        self.setup_sigma_spread_radius()
        self.set_up_hidden_to_ouput_weight()
        self.set_up_output_bias()

    def setup_center(self):
        """Setup center using clustering ,for now just randomize between 0 and 1"""
        # print("Setup center")
        for i in range(self.no_of_hidden):
            self.centroid[i] = np.random.uniform(0, 1, self.no_of_input)

    def setup_sigma_spread_radius(self):
        # print("Setup Sigma spread radius")
        for i in range(self.no_of_hidden):
            center = self.centroid[i]
            self.sigma[i] = self.set_up_sigma_for_center(center)
            # print("Sigma i",i, self.sigma[i])

    def set_up_sigma_for_center(self, center):
        # print("Get sigma for center")
        p = self.no_of_hidden / 3
        sigma = 0
        distances = [0 for i in range(self.no_of_hidden)]
        for i in range(self.no_of_hidden):
            distances[i] = self.euclidean_distance(center, self.centroid[i])
            # print("Distance ", i, distances[i])
        sum = 0
        for i in range(int(p)):
            nearest = self.get_smallest_index(distances)
            distances[nearest] = float("inf")

            neighbour_centroid = self.centroid[nearest]
            for j in range(len(neighbour_centroid)):
                sum += (center[j] - neighbour_centroid[j]) ** 2

        sigma = sum / p
        sigma = math.sqrt(sigma)
        #return random.uniform(0, 1) * 6
        return sigma

    @staticmethod
    def euclidean_distance( x, y):
        return np.linalg.norm(x-y)

    @staticmethod
    def get_smallest_index( distances):
        min_index = 0
        for i in range(len(distances)):
            if (distances[min_index] > distances[i]):
                min_index = i
        return min_index

    def set_up_hidden_to_ouput_weight(self):
        print("Setup hidden to output weight")
        self.hidden_to_output_weight = np.random.uniform(0, 1, (self.no_of_hidden, self.no_of_output))

        print("Hiden to output weight ", self.hidden_to_output_weight)

    def set_up_output_bias(self):
        print("Setup output bias")
        self.output_bias = np.random.uniform(0, 1, self.no_of_output)

    # train n iteration
    def train(self, n):
        for i in range(n):
            error = self.pass_one_epoch()
            print("Iteration ", i, " Error ", error)

        return error

    # Train an epoch and return total MSE
    def pass_one_epoch(self):
        # print("Pass one epoch")
        all_error = 0
        all_index = []
        for i in range(len(self.data.patterns)):
            all_index.append(i)
        # print("All index ",all_index)

        for i in range(len(self.data.patterns)):
            random_index = (int)(random.uniform(0, 1) * len(all_index))
            # print("Random index ",random_index, " Len ", len(all_index))
            """Get a random pattern to train"""
            pattern = self.data.patterns[random_index]
            del all_index[random_index]

            input = pattern.input
            self.actual_target_values = pattern.output
            self.pass_input_to_network(input)

            error = self.get_error_for_pattern()
            all_error += error
            self.gradient_descent()

        all_error = all_error / (len(self.data.patterns))
        return all_error

    def pass_input_to_network(self, input):
        self.input = input
        self.pass_to_hidden_node()
        self.pass_to_output_node()

    def pass_to_hidden_node(self):
        # print("Pass to hidden node")
        self.hidden_output = np.zeros(self.no_of_hidden)
        for i in range(len(self.hidden_output)):
            euclid_distance = self.euclidean_distance(self.input, self.centroid[i]) ** 2
            self.hidden_output[i] = math.exp(- (euclid_distance / (2 * self.sigma[i] * self.sigma[i])))

            # print("Hdiden node output ",self.hidden_output)

    def pass_to_output_node(self):
        # print("Pass to output node")
        self.output = [0 for i in range(self.no_of_output)]
        total = 0
        for i in range(self.no_of_output):
            output_value = 0
            for j in range(self.no_of_hidden):
                self.output[i] += self.hidden_to_output_weight[j][i] * self.hidden_output[j]
        """Normalize """
        for i in range(self.no_of_output):
            total += self.output[i]
        for i in range(self.no_of_output):
            if (self.output[i] != 0):
                self.output[i] = self.output[i] / total
        self.total = total

    # Compute error for the pattern
    def get_error_for_pattern(self):
        error = 0
        for i in range(len(self.output)):
            error += (self.actual_target_values[i] - self.output[i]) ** 2
        return error

    # Weight update by gradient descent algorithm
    def gradient_descent(self):
        # compute the error of output layer
        self.mean_error = 0
        self.error_of_output_layer = [0 for i in range(self.no_of_output)]
        for i in range(self.no_of_output):
            self.error_of_output_layer[i] = (float)(self.actual_target_values[i] - self.output[i])
            e = (float)(self.actual_target_values[i] - self.output[i]) ** 2 * 0.5
            self.mean_error += e

        # Adjust hidden to output weight
        for o in range(self.no_of_output):
            for h in range(self.no_of_hidden):
                delta_weight = self.learningRate * self.error_of_output_layer[o] * self.hidden_output[h]
                self.hidden_to_output_weight[h][o] += delta_weight

        # For bias
        for o in range(self.no_of_output):
            delta_bias = self.learningRate * self.error_of_output_layer[o]
            self.output_bias[o] += delta_bias

        # Adjust center , input to hidden weight
        for i in range(self.no_of_input):
            for j in range(self.no_of_hidden):
                summ = 0
                for p in range(self.no_of_output):
                    summ += self.hidden_to_output_weight[j][p] * (self.actual_target_values[p] - self.output[p])

                second_part = (float)((self.input[i] - self.centroid[j][i]) / math.pow(self.sigma[j], 2))
                delta_weight = (float)(self.learningRate * self.hidden_output[j] * second_part * summ)
                self.centroid[j][i] += delta_weight

        # Adjust sigma and spread radius
        for i in range(self.no_of_input):
            for j in range(self.no_of_hidden):
                summ = 0
                for p in range(self.no_of_output):
                    summ += self.hidden_to_output_weight[j][p] * (self.actual_target_values[p] - self.output[p])

                second_part = (float)(
                    (math.pow((self.input[i] - self.centroid[j][i]), 2)) / math.pow(self.sigma[j], 3));
                delta_weight = (float)(0.1 * self.learningRate * self.hidden_output[j] * second_part * summ);
                self.sigma[j] += delta_weight
        return self.mean_error

    def get_accuracy_for_training(self):
        correct = 0
        for i in range(len(self.data.patterns)):
            pattern = self.data.patterns[i]
            self.pass_input_to_network(pattern.input)
            n_output = self.output
            act_output = pattern.output
            n_neuron = self.get_fired_neuron(n_output)
            a_neuron = self.get_fired_neuron(act_output)

            if n_neuron == a_neuron:
                correct += 1
        accuracy = (float)(correct / len(self.data.patterns)) * 100
        return accuracy

    def get_fired_neuron(self, output):
        max = 0
        for i in range(len(output)):
            if (output[i] > output[max]):
                max = i
        return max

### Teste do RBF

In [3]:
p1 = Pattern(1, [0, 0], [1, 0])
p2 = Pattern(2, [0, 1], [0, 1])
p3 = Pattern(3, [1, 0], [0, 1])
p4 = Pattern(4, [1, 1], [1, 0])

patterns = [p1, p2, p3, p4]
classLabels = ['0', '1']
data = Data(patterns, classLabels)
rbf = RBFNetwork(2, 6, 2, data)
mse = rbf.train(1500)
accuracy = rbf.get_accuracy_for_training()
print("Total accuracy is ", accuracy)
print("Last MSE ",mse)

Setup hidden to output weight
Hiden to output weight  [[0.78606254 0.91909265]
 [0.17152948 0.23860971]
 [0.24219289 0.26454683]
 [0.83155311 0.83415991]
 [0.19370434 0.09976309]
 [0.46305319 0.53439819]]
Setup output bias
Iteration  0  Error  0.5761377283032174
Iteration  1  Error  0.5170700836754278
Iteration  2  Error  0.5699409224062688
Iteration  3  Error  0.5081394406593814
Iteration  4  Error  0.5741901804694542
Iteration  5  Error  0.514827282035097
Iteration  6  Error  0.503522373240788
Iteration  7  Error  0.5371396348778947
Iteration  8  Error  0.4766343166017795
Iteration  9  Error  0.5349656022178326
Iteration  10  Error  0.6123405712871463
Iteration  11  Error  0.47380506188752514
Iteration  12  Error  0.5951539461221195
Iteration  13  Error  0.5717242696131064
Iteration  14  Error  0.5962790020733304
Iteration  15  Error  0.5066809982919455
Iteration  16  Error  0.5328220635930047
Iteration  17  Error  0.6428257422753796
Iteration  18  Error  0.5959219176754708
Iteration

Iteration  234  Error  0.41680818978623163
Iteration  235  Error  0.6252122756238512
Iteration  236  Error  0.425050175201532
Iteration  237  Error  0.31672408905947963
Iteration  238  Error  0.20840387739054692
Iteration  239  Error  0.5597544575844021
Iteration  240  Error  0.5600117044866265
Iteration  241  Error  0.3170432265720087
Iteration  242  Error  0.4597877097425595
Iteration  243  Error  0.41766414175371824
Iteration  244  Error  0.31719142184833116
Iteration  245  Error  0.4176640037716549
Iteration  246  Error  0.10835741524250429
Iteration  247  Error  0.35153205940285537
Iteration  248  Error  0.21673878194388876
Iteration  249  Error  0.4179275555680442
Iteration  250  Error  0.41792755092299994
Iteration  251  Error  0.41792754627713147
Iteration  252  Error  0.4179275416319854
Iteration  253  Error  0.46001397534883814
Iteration  254  Error  0.1083793755493436
Iteration  255  Error  0.3174679895203521
Iteration  256  Error  0.31746598231215817
Iteration  257  Error  

Iteration  449  Error  0.4233795054389573
Iteration  450  Error  0.21168975131642093
Iteration  451  Error  0.46217837366420567
Iteration  452  Error  0.3204021829518864
Iteration  453  Error  0.250447516621119
Iteration  454  Error  0.42351507365025126
Iteration  455  Error  0.21175753543072925
Iteration  456  Error  0.5322021527729724
Iteration  457  Error  0.42351498708674173
Iteration  458  Error  0.4235149833683433
Iteration  459  Error  0.32044254412382533
Iteration  460  Error  0.3535332298112542
Iteration  461  Error  0.35354829786088093
Iteration  462  Error  0.1087092593567948
Iteration  463  Error  0.42364301363941376
Iteration  464  Error  0.35356313195435923
Iteration  465  Error  0.5324246423478088
Iteration  466  Error  0.532422492158605
Iteration  467  Error  0.32056821542242364
Iteration  468  Error  0.6355562589899086
Iteration  469  Error  0.2118520844869091
Iteration  470  Error  0.21185208356537358
Iteration  471  Error  0.3205660984781118
Iteration  472  Error  0.

Iteration  680  Error  6.096892164710419e-22
Iteration  681  Error  0.6382127790846837
Iteration  682  Error  0.2182303526320447
Iteration  683  Error  0.6382125810363821
Iteration  684  Error  0.14105588727170754
Iteration  685  Error  0.2127454382313132
Iteration  686  Error  0.4629327457345736
Iteration  687  Error  0.46293935121658913
Iteration  688  Error  0.3537931929355318
Iteration  689  Error  6.25186070280045e-22
Iteration  690  Error  0.10915267354772466
Iteration  691  Error  0.32191858785261096
Iteration  692  Error  0.4255361765749365
Iteration  693  Error  0.2182943070056932
Iteration  694  Error  0.3219118295578959
Iteration  695  Error  0.32190958424510613
Iteration  696  Error  1.0737034825741784e-21
Iteration  697  Error  0.21276795911660273
Iteration  698  Error  0.10913940984028894
Iteration  699  Error  0.46294318686243663
Iteration  700  Error  0.21829021900025847
Iteration  701  Error  0.32191686870892877
Iteration  702  Error  0.2182768964675249
Iteration  703 

Iteration  911  Error  0.21302328717637956
Iteration  912  Error  0.1405900176306627
Iteration  913  Error  0.322489590953617
Iteration  914  Error  0.6390774834379938
Iteration  915  Error  0.25005407080663844
Iteration  916  Error  7.9731432871799945e-22
Iteration  917  Error  0.21893543503933138
Iteration  918  Error  0.4260565361513845
Iteration  919  Error  0.10946415509706076
Iteration  920  Error  0.140578370782992
Iteration  921  Error  0.35360347287586535
Iteration  922  Error  0.353597833728883
Iteration  923  Error  0.5355624507289841
Iteration  924  Error  0.14055513605585704
Iteration  925  Error  0.4320329915267384
Iteration  926  Error  0.3225318699664712
Iteration  927  Error  0.5355672504324162
Iteration  928  Error  0.1094892236443269
Iteration  929  Error  0.42607558117347216
Iteration  930  Error  0.2189712939658104
Iteration  931  Error  0.35359373543356853
Iteration  932  Error  0.46308966575510335
Iteration  933  Error  0.2130423449023789
Iteration  934  Error  0

Iteration  1132  Error  0.43229306291386815
Iteration  1133  Error  0.219171467921982
Iteration  1134  Error  0.5665779215816612
Iteration  1135  Error  0.5358156988298425
Iteration  1136  Error  0.3534608960963028
Iteration  1137  Error  0.21919194442323037
Iteration  1138  Error  0.3534573862141748
Iteration  1139  Error  0.21311474780515943
Iteration  1140  Error  0.4323142186738901
Iteration  1141  Error  0.5665697500888532
Iteration  1142  Error  0.213115865109158
Iteration  1143  Error  0.3227206976972204
Iteration  1144  Error  0.21311583370786596
Iteration  1145  Error  0.3534493095013108
Iteration  1146  Error  1.024288915602081e-21
Iteration  1147  Error  0.2131169967826266
Iteration  1148  Error  0.14032551537913787
Iteration  1149  Error  0.21311814548639335
Iteration  1150  Error  0.24994773644534735
Iteration  1151  Error  0.3227456561179911
Iteration  1152  Error  0.21311921893089159
Iteration  1153  Error  0.4262384354256822
Iteration  1154  Error  0.24994689842995726
I

Iteration  1350  Error  0.21315769743177462
Iteration  1351  Error  0.2194940503947646
Iteration  1352  Error  0.21315763740546007
Iteration  1353  Error  0.4263152723848582
Iteration  1354  Error  0.21948288942609528
Iteration  1355  Error  0.3532947116981171
Iteration  1356  Error  0.24988412262683302
Iteration  1357  Error  0.3532852788834413
Iteration  1358  Error  0.32291821677064225
Iteration  1359  Error  0.2131593322405655
Iteration  1360  Error  0.5360746900341145
Iteration  1361  Error  0.1401225632019711
Iteration  1362  Error  0.3532771278343162
Iteration  1363  Error  0.4326966185232395
Iteration  1364  Error  0.3229242871763098
Iteration  1365  Error  0.536081839908231
Iteration  1366  Error  0.6394810670234573
Iteration  1367  Error  0.10975828891904045
Iteration  1368  Error  0.4263206481089553
Iteration  1369  Error  0.4326684144595797
Iteration  1370  Error  0.10974985738440395
Iteration  1371  Error  0.10974704235019336
Iteration  1372  Error  0.1097442250487462
Iter